In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append(
    os.path.abspath(
        os.path.join(
            os.getcwd(),
            os.pardir
        )
    )
)
__package__ = "mindlogger_backend_dev.update_girder"
from datetime import date, datetime
import girder_client
import girder_connections
import json
import object_manipulation
import pandas as pd
import psycopg2
from urllib.parse import quote
import update_schema
import urllib
from urllib.request import urlopen

which_girder = "dev"
config, context, api_url = girder_connections.configuration(
    which_girder=which_girder
)
which_girder = "girder-{}".format(which_girder)

girder_connection = girder = girder_connections.connect_to_girder(
    api_url=api_url,
    authentication=(
        config[which_girder]["user"],
        config[which_girder]["password"],
        config[which_girder]["APIkey"]
    ) if "APIkey" in config[which_girder] else (
        config[which_girder]["user"],
        config[which_girder]["password"]
    )
)


In [ ]:
activites_id = update_schema.get_girder_id_by_name(
    girder,
    "Collection",
    "Activities"
)
screens_id = update_schema.get_girder_id_by_name(
    girder,
    "Collection",
    "Screens"
)
schedules_id = update_schema.get_girder_id_by_name(
    girder,
    "Collection",
    "Schedules"
)
users_ids = [user["_id"] for user in girder.get("user")]
volumes_id = update_schema.get_girder_id_by_name(
    girder,
    "Collection",
    "Volumes"
)

In [ ]:
for schedule in update_schema.ls_x_in_y(
    "Folder",
    ("Collection", schedules_id),
    girder
):
    versions = update_schema.ls_x_in_y(
        "Item",
        ("Folder", schedule['_id']),
        girder
    )
    for version in versions:
        volume = version['name'].split(schedule['name'])[0].strip()
        volume_id = update_schema.find_or_create(
            ('Folder', volume),
            ('Collection', volumes_id),
            girder
        )
        volume_schedules_folder = update_schema.find_or_create(
            ('Folder', "Schedules"),
            ('Folder', volume_id),
            girder
        )
        update_schema.mv(
            ('Folder', schedule['_id']),
            ('Folder', volume_schedules_folder),
            girder
        )
girder.delete("collection/{}".format(schedules_id))

In [ ]:
for activity in update_schema.ls_x_in_y(
    "Folder",
    ("Collection", activites_id),
    girder
):
    versions = update_schema.ls_x_in_y(
        "Item",
        ("Folder", activity['_id']),
        girder
    )
    for version in versions:
        volume = version['name'].split(activity['name'])[0].strip(" ―")
        volume_id = update_schema.find_or_create(
            ('Folder', volume),
            ('Collection', volumes_id),
            girder
        )
        update_schema.rename(
            ("Item", version["_id"]),
            version['name'].split(volume)[1].strip("― "),
            girder_connection
        )
        volume_activities_folder = update_schema.find_or_create(
            ('Folder', "Activities"),
            ('Folder', volume_id),
            girder
        )
        update_schema.mv(
            ('Folder', activity['_id']),
            ('Folder', volume_activities_folder),
            girder
        )
girder.delete("collection/{}".format(activites_id))

In [ ]:
for screens in update_schema.ls_x_in_y(
    "Folder",
    ("Collection", screens_id),
    girder
):
    volume = screens['name'].split("―")[0].strip(" ―")  
    volume_id = update_schema.find_or_create(
        ('Folder', volume),
        ('Collection', volumes_id),
        girder
    )
    update_schema.rename(
        ("Folder", screens["_id"]),
        screens['name'].split(volume)[1].strip("― "),
        girder_connection
    )
    volume_screens_folder = update_schema.find_or_create(
        ('Folder', "Screens"),
        ('Folder', volume_id),
        girder
    )
    update_schema.mv(
        ('Folder', screens['_id']),
        ('Folder', volume_screens_folder),
        girder
    )
girder.delete("collection/{}".format(screens_id))

In [ ]:
for user in users_ids:
    for user_folder in update_schema.ls_x_in_y(
        "Folder",
        ("User", user),
        girder_connection
    ):
        for item in update_schema.ls_x_in_y(
            "Item",
            ("Folder", user_folder["_id"]),
            girder_connection
        ):
            volume = item['name'][:9]
            volume_id = update_schema.find_or_create(
                ('Folder', volume),
                ('User', user),
                girder_connection
            )
            update_schema.rename(
                ("Item", item["_id"]),
                item['name'].split(volume)[1].strip(),
                girder_connection
            )
            update_schema.mv(
                ("Folder", user_folder["_id"]),
                ("Folder", volume_id),
                girder_connection
            )
        for internal_folder in update_schema.ls_x_in_y(
            "Folder",
            ("Folder", user_folder["_id"]),
            girder_connection
        ):
            for version in update_schema.ls_x_in_y(
                "Folder",
                ("Folder", internal_folder["_id"]),
                girder_connection
            ):
                volume = version['name'].split(
                    internal_folder['name']
                )[0].strip(" ―")
                volume_id = update_schema.find_or_create(
                    ('Folder', volume),
                    ('User', user),
                    girder_connection
                )
                update_schema.rename(
                    ("Folder", version["_id"]),
                    version['name'].split(volume)[1].strip("― "),
                    girder_connection
                )
                try:
                    update_schema.mv(
                        ("Folder", user_folder["_id"]),
                        ("Folder", volume_id),
                        girder_connection
                    )
                except:
                    pass # if this function persists, add code to copy relevant Folders

In [ ]:
examples = girder_client.GirderClient(
    apiUrl="https://data.kitware.com/api/v1/"
)

### Update Activities (Item to Folder) in "Volume 0"

In [ ]:
volume_name = "Volume 0"
activities_id = girder_connection.get(
    "&".join([
        "folder?parentType=folder",
        "parentId={}".format(
            girder_connection.get(
                "&".join([
                    "folder?parentType=collection",
                    "parentId={}".format(volumes_id),
                    "text={}".format(volume_name)
                ])
            )[0]["_id"]
        ),
        "name={}".format(
            "Activities"
        )
    ])
)[0]["_id"] # get "Activities" _id within Volume

In [ ]:
activity_ids = [
    a["_id"] for a in girder_connection.get(
        "&".join([
            "folder?parentType=folder",
            "parentId={}".format(
                activities_id
            )
        ])
    )
] # get _ids of each Activity

In [ ]:
for activity in activity_ids:
    [
        update_schema.move_item_to_folder(
            version["_id"],
            girder_connection
        ) for version in girder_connection.get(
            "&".join([
                "item?folderId={}".format(activity),
            ])
        )
    ]